In [117]:
import numpy as np
import pandas as pd
title = 'woplus'
path = '../../sources/data'

In [118]:
df_net = pd.read_table('{}/dataset_2015_filter_net.txt'.format(path), sep='|')
df_net_piv = df_net.pivot(index='imsi', columns='mon', values='nettype')
df_net_piv.columns = ['mon01','mon02','mon03','mon04','mon05','mon06','mon07','mon08','mon09','mon10','mon11','mon12']

In [146]:
das = df_net_piv.loc[:,'mon01']
a = 0.01
tmp = pd.value_counts(das)/das.shape[0]
cat = list(tmp.index[tmp > a])
cat

['2G', '3G']

In [141]:
tmp

2G    0.625362
3G    0.374638
Name: mon01, dtype: float64

In [ ]:
# test 测试 对list排序
# cat.sort()
# cat.sort(reverse=True)
# cat

In [147]:
# cat = ['2G', '3G'] # 可以自己定义，否则不能保证顺序每次都一致
cat.sort()   # 改良：使用cat前对cat进行一次排序，保证顺序一致
# das.astype('category').cat.set_categories(['2G', '3G']).cat.rename_categories(1+np.arange(len(['2G', '3G']))).astype("float").fillna(0).values
# das.astype('category').cat.set_categories(['2G', '3G']).cat.rename_categories(1+np.arange(len(['2G', '3G']))).astype("float").fillna(0).values[:, np.newaxis]
df_net_cat = np.transpose([das.astype('category').cat.set_categories(cat).cat.rename_categories(1+np.arange(len(cat))).astype('float').fillna(0).values])
df_net_cat

array([[ 2.],
       [ 1.],
       [ 1.],
       ..., 
       [ 2.],
       [ 2.],
       [ 1.]])

In [148]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(n_values = len(cat)+1, sparse = False)
xbin = enc.fit_transform(df_net_cat)[:,1:]
dabin = pd.DataFrame(xbin, columns = ["{}_{}".format(das.name, x) for x in cat], index = das.index)
dabin.head()

,mon01_2G,mon01_3G
imsi,,
00002ec9f6870677ca801c213ce87c02,0,1
0000be0fbdbe7f8ff47f7ec3e077e489,1,0
000166f550781551e1eb11cfcd3f4267,1,0
0001a48e7fca2e7e3279a5f275525cdb,1,0
0001b4ebaa72f0e4c2e79a0791bf3002,1,0


In [145]:
# tmp[tmp <= a].sum()

0.37463750838651727

In [150]:
if(tmp[tmp <= a].sum() > a):
    dabin = pd.concat([dabin, pd.DataFrame({"{}_Others".format(das.name):das.notnull()-dabin.sum(axis = 1)})], axis = 1)
if(dabin.shape[1] == 2):
    dabin = pd.DataFrame({das.name: xbin[:,0]}, index = das.index)
dabin.head()

,mon01
imsi,
00002ec9f6870677ca801c213ce87c02,0
0000be0fbdbe7f8ff47f7ec3e077e489,1
000166f550781551e1eb11cfcd3f4267,1
0001a48e7fca2e7e3279a5f275525cdb,1
0001b4ebaa72f0e4c2e79a0791bf3002,1
